<a href="https://colab.research.google.com/github/RaianaSeixas/Paper-1-Final-Results/blob/master/Stats_2000_Step_ALL3_D_30_0_25_0_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install pyade-python

In [14]:
!pip install PyFDE

In [15]:
import pyade.ilshade 
import pyade.sade 
import pyade.jso
import pyade.shade
import pyade.jade 
import pyfde
import numpy as np
import pandas as pd
import time

In [16]:
!git clone https://github.com/ucfilho/Raianars_July_2020 #clonar do Github
%cd Raianars_July_2020

Cloning into 'Raianars_July_2020'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 147 (delta 12), reused 0 (delta 0), pack-reused 123
Receiving objects: 100% (147/147), 204.22 KiB | 397.00 KiB/s, done.
Resolving deltas: 100% (74/74), done.
/content/Raianars_July_2020/ANN_read_matrix/Raianars_July_2020


In [17]:
import Go2Ann
import de_soma_Insight
import de_bounds_matriz
from AvaliaX import AvaliaX
from GeraIndices import GeraIndices

In [18]:
import Function
import Go2MinMax

In [19]:
ITE=2000 #10 # PONTOS quantas vezes vai alteral NPAR 
NPAR=100 # NUMBER OF PARTICLES
dim=30
RUNS=50
fchoice='Step'  
Fun=Function.Step

MIN,MAX=Go2MinMax.Intervalo(dim,fchoice)
bounds = [(0,0)]*dim

for i in range(dim):
  bounds[i] = (MIN[i],MAX[i])

start_time = time.time() 

In [20]:
############################### INPUT INFORMATION (FRANNK)
Fc=0.1 # fator de correcao parametros da ANN
its=1 #10 # quantas vezes vai usar o NPAR antes de alterar de novo
mut=0.25
crossp=0.25
Fi=mut;CRi=crossp;# para impressão
PAR=len(MAX) #NUM DE PARAMETROS A SER OTIMIZADOS

In [21]:
# JADE
algorithm = pyade.jso
BEST_AGENTS=[]
for i in range(RUNS):
  ANSWER_AGENTS=[]
  solver = pyfde.JADE(Fun, n_dim=dim, n_pop=NPAR, limits=bounds, minimize=True)
  solver.p = 0.05
  solver.c = 0.1
  solver.cr = 0.5
  solver.f = 0.5
  best, fit = solver.run(n_it=ITE)
  for x in best:
    ANSWER_AGENTS.append(x)
  ANSWER_AGENTS.append(fit)
  BEST_AGENTS.append(ANSWER_AGENTS)
BEST_AGENTS=np.array(BEST_AGENTS)
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()
nome='JADE_Fo-'+str(0)+'_CRo-'+str(0)
print(nome,'NPAR',NPAR,'Iterações=',ITE,'Dim=',dim ,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print( '\n')
print('MÉDIA: ',MEAN[-1])
print('MIN __:',MINFO)
print('MAX__: ',MAXFO)
print('STD __:',STD)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)

JADE_Fo-0_CRo-0 NPAR 100 Iterações= 2000 Dim= 30 Min/Max= -100 / 100 _ Step


MÉDIA:  0.0
MIN __: -0.0
MAX__:  -0.0
STD __: 0.0
Tempo Médio= 2.241 segundos ---
50 Rodadas

BEST:  [-0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5
 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5
 -0.5 -0.5 -0. ]


In [22]:
# FRANNK START HERE !!!!!

In [23]:
!git clone https://github.com/ucfilho/ANN_read_matrix #clonar do Github
%cd ANN_read_matrix

Cloning into 'ANN_read_matrix'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.
/content/Raianars_July_2020/ANN_read_matrix/Raianars_July_2020/ANN_read_matrix


In [24]:
ANN_F=pd.read_csv('ANN_INSIGHT_Valor_F.csv')
ANN_CR=pd.read_csv('ANN_INSIGHT_CR.csv')
syn0_F,syn1_F,X_max_F,X_min_F=Go2Ann.GetMatriz(ANN_F)
syn0_CR,syn1_CR,X_max_CR,X_min_CR=Go2Ann.GetMatriz(ANN_CR)
fields = ['DI RELATIVO','FRAC Its','Fo','CRo']

#TOTAL=7*dim*its*ITE
TOTAL=10*dim*its*ITE
TOTAL=int(TOTAL/2)   # CONFERIR SE ESTE EH O MELHOR LUGAR

dimensions=len(MAX)
start_time = time.time() #INICIALIZA CONTAGEM DO TEMPO

MAX_P=np.zeros(ITE) # ATENCAO para variar F e Cr MAX_P passa a ser 3 vezes maior
MIN_P=np.zeros(ITE) # ATENCAO para variar F e Cr MIN_P passa a ser 3 vezes maior

for i in range(dim):
  MAX_P[i]=MAX[i] 
  MIN_P[i]=MIN[i]

rodadas=1
for i in range(rodadas):
  SOMA=0
  pop = np.random.rand(NPAR, dimensions)
  diff=np.zeros(PAR)
  for ii in range(PAR):
    diff[ii] = MAX[ii]-MIN[ii]
  X = MIN + pop * diff
    
  PONTOS=ITE # DIVISOES DA POPULACAO
start_time = time.time()
BEST_AGENTS=[]
MAT_stat=[]
for i in range(RUNS):
    MAT=[] 
    mut=Fi
    crossp=CRi
    SOMA=0
    SOMAf=0
    start_ite = time.time()
    delta_time=(start_ite-start_time)/60
    print('ite=',i,'time=',delta_time)
    pop = np.random.rand(NPAR, dim)
    diff=np.zeros(PAR)
    for ii in range(PAR):
      diff[ii] = MAX[ii]-MIN[ii]
    X = MIN + pop * diff
    Xf = MIN + pop * diff
    
    DIo=1e99
  
    CRo=crossp;Fo=mut;
    Fitness = np.asarray([Fun(ind) for ind in X])

    XY,BEST_XY,BESTo,FOBESTo=AvaliaX(X,Fitness)
    nrow,ncol=X.shape

    QUANT=17;MAT_INDo=np.zeros((1,QUANT)); # quantos indices esta fazendo
    MAT_INDo[0,5]=nrow  # VALOR NP
    MAT_INDo[0,6]=Fo    # VALOR Fo   
    MAT_INDo[0,7]=CRo   # VALOR CRo
  
    MAT_INDo=GeraIndices(X,BESTo,FOBESTo,DIo,MAT_INDo,SOMA,TOTAL,syn0_F,
                        syn1_F,X_max_F,X_min_F,syn0_CR,syn1_CR,X_max_CR,
                        X_min_CR,Fc,fields,Fun)
  
    DIo=np.copy(MAT_INDo[0,0])
    MAT_INDo[0,15]=Fo    # VALOR Fo   
    MAT_INDo[0,16]=CRo   # VALOR CRo
    #ind_crossp=ITE-1

    
    for k in range(ITE):
      X,BEST,FOBEST,XY,BEST_XY,SOMA= de_soma_Insight.de(MAX,MIN, mut, crossp,NPAR,its,Fun,X,SOMA,TOTAL)
      #Xf,BESTf,FOBESTf,XYf,BEST_XYf,SOMAf= de_soma_Insight.de(MAX,MIN, Fi, CRi, NPAR,its,Fun,Xf,SOMAf,TOTAL)

      
      MAT_IND=GeraIndices(X,BESTo,FOBESTo,DIo,MAT_INDo,SOMA,TOTAL,syn0_F,
                          syn1_F,X_max_F,X_min_F,syn0_CR,syn1_CR,X_max_CR,
                          X_min_CR,Fc,fields,Fun)
    
      BESTo=np.copy(BEST);FOBESTo=np.copy(FOBEST);
      MAT.append(MAT_IND.ravel())
    
      MAT_INDo[0,6]=MAT_IND[0,15]   # VALOR Fo   
      MAT_INDo[0,7]=MAT_IND[0,16]   # VALOR CRo
      mut=MAT_IND[0,15]
      crossp=MAT_IND[0,16]

      Fitness = np.asarray([Fun(ind) for ind in X])
      #Fitnessf = np.asarray([Fun(ind) for ind in Xf])
      Maior=np.argmax(Fitness)
      #Maiorf=np.argmax(Fitnessf)
      Menor=np.argmin(Fitness)
      #Menorf=np.argmin(Fitnessf)

      #X[Maior,:]=np.copy(Xf[Menorf,:])
      #Xf[Maiorf,:]=np.copy(X[Menor,:])
      
    BEST_AGENTS.append(BEST_XY)
    MAT_stat.append(MAT)
    #print("GlobalParam e Solucao:", BEST,"Funcao=",FOBEST)
    
############################## Cálculo dos Parâmetros de Avaliação   
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()

print()
nome='_ANN_Div3x4_Fo-'+str(Fi)+'_CRo-'+str(CRi)+'_D-'+str(dim)+'_Interv-'+str(ITE)
print('DE_POP_VAR_F_CR','TOTAL=',TOTAL,'Iterações=',ITE*its,'Dim=', PAR,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print(nome, '\n')
print('MÉDIA: ',MEAN[-1])
print('MIN __:',MINFO)
print('MAX__: ',MAXFO)
print('STD __:',STD)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)

ite= 0 time= 0.00016983350118001303


/content/Raianars_July_2020/GeraIndices.py:57: RuntimeWarning: invalid value encountered in double_scalars
  if abs(MAT_IND[0,8]/MAT_IND[0,8])>1:
/content/Raianars_July_2020/GeraIndices.py:60: RuntimeWarning: invalid value encountered in double_scalars
  MAT_IND[0,10]=MAT_IND[0,8]/MAT_IND[0,8]
/content/Raianars_July_2020/GeraIndices.py:62: RuntimeWarning: invalid value encountered in double_scalars
  if abs(MAT_IND[0,9]/MAT_IND[0,9])>1: # tem que pegar primeira
/content/Raianars_July_2020/GeraIndices.py:65: RuntimeWarning: invalid value encountered in double_scalars
  MAT_IND[0,11]=MAT_IND[0,9]/MAT_IND[0,9] # tem que pegar do anterior


ite= 1 time= 2.349908904234568
ite= 2 time= 4.688973450660706
ite= 3 time= 7.048907601833344
ite= 4 time= 9.382045288880667
ite= 5 time= 11.725668370723724
ite= 6 time= 14.116401219367981
ite= 7 time= 16.507688558101655
ite= 8 time= 18.87537502447764
ite= 9 time= 21.268607147534688
ite= 10 time= 23.65127775669098
ite= 11 time= 26.04885079463323
ite= 12 time= 28.43492655356725
ite= 13 time= 30.80465977191925
ite= 14 time= 33.15918777783712
ite= 15 time= 35.55995012521744
ite= 16 time= 37.91811483303706
ite= 17 time= 40.30331554412842
ite= 18 time= 42.688700811068216
ite= 19 time= 45.08157906134923
ite= 20 time= 47.46709367036819
ite= 21 time= 49.82454326550166
ite= 22 time= 52.21990067958832
ite= 23 time= 54.59723608891169
ite= 24 time= 56.97707460323969
ite= 25 time= 59.36908361514409
ite= 26 time= 61.7585194905599
ite= 27 time= 64.14752160708109
ite= 28 time= 66.54265495141347
ite= 29 time= 68.8951118628184
ite= 30 time= 71.28721449375152
ite= 31 time= 73.67386693159739
ite= 32 time= 

In [25]:
#DE_ tradicional

!git clone https://github.com/ucfilho/Raianars_July_2020 #clonar do Github
%cd Raianars_July_2020

mut=0.5
crossp=0.5

SOMA=0
start_time = time.time()
BEST_AGENTS=[]
MAT_stat=[]

for i in range(RUNS):

    pop = np.random.rand(NPAR, dim)
    diff=np.zeros(dim)
    for ii in range(dim):
      diff[ii] = MAX[ii]-MIN[ii]
    X = MIN + pop * diff

    SOMA=0

    for k in range(ITE):
      X,BEST,FOBEST,XY,BEST_XY,SOMA= de_soma_Insight.de(MAX,MIN, mut, crossp,dim,its,Fun,X,SOMA,TOTAL)

    start_run = (time.time()-start_time)/60
    print('run=',i,'total=50','Fobj=',FOBEST,'Time=',start_run)  
    BEST_AGENTS.append(BEST_XY)

Cloning into 'Raianars_July_2020'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 147 (delta 12), reused 0 (delta 0), pack-reused 123
Receiving objects: 100% (147/147), 204.22 KiB | 518.00 KiB/s, done.
Resolving deltas: 100% (74/74), done.
/content/Raianars_July_2020/ANN_read_matrix/Raianars_July_2020/ANN_read_matrix/Raianars_July_2020
run= 0 total=50 Fobj= 1.3399850181068632e-28 Time= 0.34809781710306803
run= 1 total=50 Fobj= 7.635927043506513e-30 Time= 0.6974609057108562
run= 2 total=50 Fobj= 8.38472860588427e-30 Time= 1.0445255041122437
run= 3 total=50 Fobj= 4.767061798347286e-30 Time= 1.3903652985890707
run= 4 total=50 Fobj= 1.8495090441939503e-29 Time= 1.7360013008117676
run= 5 total=50 Fobj= 3.710111444867571e-30 Time= 2.0845688144365946
run= 6 total=50 Fobj= 3.3064365285240065e-30 Time= 2.4369940121968585
run= 7 total=50 Fobj= 4.5821725236861115e-30 Time= 2.7850443641344707
run

In [26]:
############################## Cálculo dos Parâmetros de Avaliação   
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()

nome='DE_STAT_Fo-'+str(mut)+'_CRo-'+str(crossp)+'_D-'+str(dim)+'_Interv-'+str(ITE)
print('DE_','TOTAL=',TOTAL,'Iterações=',ITE*its,'Dim=', dim,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print(nome, '\n')
#print('BEST: ',BEST)
#print('MÉDIA: ',MEAN)
print('MÉDIA: ',MEAN[-1])
print('MIN __:',MINFO)
print('MAX__: ',MAXFO)
print('STD __:',STD)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)

DE_ TOTAL= 300000 Iterações= 2000 Dim= 30 Min/Max= -100 / 100 _ Step
DE_STAT_Fo-0.5_CRo-0.5_D-30_Interv-2000 

MÉDIA:  3.9185987539513337e-29
MIN __: 3.3064365285240065e-30
MAX__:  2.488640451818521e-28
STD __: 4.517712070956183e-29
Tempo Médio= 20.866 segundos ---
50 Rodadas

BEST:  [-5.00000000e-01 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01
 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01
 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01
 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01
 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01
 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01
 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01
 -5.00000000e-01 -5.00000000e-01  3.30643653e-30]
